In [1]:
import os

In [2]:
%pwd

'/home/ilia/stream_proj/fecal-chicken/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/ilia/stream_proj/fecal-chicken'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManconager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [13]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size


In [15]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f'{filename} downloaded with following info: \n {headers}')
        else:
            logger.info(f'File already exists with the size of {get_size(Path(self.config.local_data_file))}')
   
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_file:
            zip_file.extractall(unzip_path)

In [16]:
try:
    configMGR = ConfigurationManconager()
    data_ingestion_config = configMGR.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2023-08-24 14:46:20,992: INFO: common: yaml file <_io.TextIOWrapper name='config/config.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2023-08-24 14:46:21,005: INFO: common: yaml file <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2023-08-24 14:46:21,006: INFO: common: created directory at artifacts]
[2023-08-24 14:46:21,007: INFO: common: created directory at artifacts/data_ingestion]
[2023-08-24 14:46:48,637: INFO: 2165489684: artifacts/data_ingestion/data.zip downloaded with following info: 
 x-amz-id-2: XuzlgFtL0qe/HI88nenMazBsD/z45J48j46uSXBa3AkOONb5gaBHhX4Naws65deLctGzRJbcBwY=
x-amz-request-id: 2H463ZAA839HYC6Z
Date: Thu, 24 Aug 2023 07:46:24 GMT
Last-Modified: Thu, 24 Aug 2023 06:46:36 GMT
ETag: "efd97489dac63b9efc47b39b84ad540a"
x-amz-server-side-encryption: AES256
Accept-Ranges: bytes
Content-Type: application/zip
Server: AmazonS3
Content-Length: 11621224
Connection: close

]
